# Assignment - Segmenting and Clustering Neighborhoods in Toronto

In [21]:
import pandas as pd
from bs4 import BeautifulSoup


I will use the library BeautifulSoup to get my data

In [139]:
#downloading the site

!wget -O wiki_postal.html https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


--2019-11-10 21:38:32--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79033 (77K) [text/html]
Saving to: ‘wiki_postal.html’

100%[======================================>] 79,033       472KB/s   in 0.2s   

2019-11-10 21:38:33 (472 KB/s) - ‘wiki_postal.html’ saved [79033/79033]



In [140]:
#reading the file
with open('wiki_postal.html', 'r') as outfile:
    raw_site = outfile.read()
    
soup = BeautifulSoup(raw_site, 'html.parser')

#we are interested in the first table of the site
table = soup.find_all('table')[0] 
df_ = pd.read_html(str(table))[0]
df_.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [141]:
#removing row without Borough assigned
df_=df_[df_['Borough']!='Not assigned']
df_.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [142]:
#check if there is still any duplicate postcode
any(df_.duplicated(subset=['Postcode']))

True

In [143]:
#merging rows with same postcode by joining Neighbourhood
df_=df_.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [144]:
#check if there is still any duplicate postcode
any(df_.duplicated(subset=['Postcode']))

False

In [145]:
#check if there are rows with Neighbourhood not assigned
any(df_['Neighbourhood']=='Not assigned')

True

In [146]:
#if Neighbourhood is not assigned, use the borough value
df_['Neighbourhood'] = df_.apply(lambda x: x['Neighbourhood'] if x['Neighbourhood']!= 'Not assigned' else x['Borough'],axis=1)
df_.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [147]:
#check if there are rows with Neighbourhood not assigned
any(df_['Neighbourhood']=='Not assigned')

False

In [149]:
df_.shape

(103, 3)

In [151]:
df_.to_csv('canada_postal_code.csv')

In [158]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [162]:
df_ = pd.read_csv('canada_postal_code.csv')
df_ = pd.merge(df_,geo_data,left_on='Postcode',right_on='Postal Code')
df_.head(10)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


In [167]:
df_.drop('Unnamed: 0',axis=1,inplace=True)
df_.head(10)

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848
